In [ ]:
!pip install pyspark
!pip install -q google-cloud-storage==1.43.0
from inverted_index_gcp import *
import pyspark


In [ ]:
import sys
from collections import Counter, OrderedDict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

signal.signal(signal.SIGALRM, handler)
signal.alarm(AUTH_TIMEOUT)

try:
   auth.authenticate_user()
except:
   pass

In [ ]:
# inverted text upload
bucket_name = "bucket324955145"
file_path = "postings_gcp/text/postings_gcp_text_inverted_text_v1.pkl"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
contents = blob.download_as_bytes()
inverted_text = pickle.loads(contents)

In [ ]:
# inverted title upload
bucket_name = "bucket324955145"
file_path = "postings_gcp/title/postings_gcp_title_inverted_title_v1.pkl"
storage_client = storage.Client()
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(file_path)
contents = blob.download_as_bytes()
inverted_title = pickle.loads(contents)

In [ ]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links",
                    "may", "first", "see", "history", "people", "one", "two",
                    "part", "thumb", "including", "second", "following",
                    "many", "however", "would", "became"]

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)

In [ ]:
# doc len
text_doc_len = inverted_text.doc_len.copy()
title_doc_len = inverted_title.doc_len.copy()

In [ ]:
# idf
text_idf = inverted_text.idf
title_idf = inverted_title.idf

In [ ]:
# idf bm 25
# text_idf_bm25 = inverted_text.get_idf_bm25()
# title_idf_bm25 = inverted_title.get_idf_bm25()
text_idf_bm25 = inverted_text.idf_bm25
title_idf_bm25 = inverted_title.idf_bm25

In [ ]:
# advl
# text_avdl = inverted_text.get_avdl()
# title_avdl = inverted_title.get_avdl()
text_avdl = inverted_text.avdl
title_avdl = inverted_title.avdl

In [ ]:
# check to see what values we can work with now:
print(len(inverted_text.doc_len.copy()))
print(len(inverted_title.doc_len.copy()))
print(len(inverted_text.idf))
print(len(inverted_title.idf))
print(len(text_idf_bm25))
print(len(title_idf_bm25))
print(text_avdl)
print(title_avdl)

6347138
6340943
495532
103028
495532
103028
319.6134407350211
2.5288798527285294


In [108]:
k1 = 1.2
k3 = 0
b = 0.5

In [109]:
# testing:

def retrieve_posting_list_title(query_word: str, bucket_name: str):
    return inverted_title.read_a_posting_list(base_dir='.', w=query_word, bucket_name=bucket_name)

def retrieve_posting_list_text(query_word: str, bucket_name: str):
    return inverted_text.read_a_posting_list(base_dir='.', w=query_word, bucket_name=bucket_name)

def calculate_bm25_per_tf(token, doc_id_tf, inverted):
    doc_id, tf = doc_id_tf
    B = 1 - b + b * inverted_title.doc_len.get(doc_id)
    return ((k1 + 1) / (B * k1 + tf)) * ((inverted_title.corpus_size + 1) /inverted_title.idf_bm25[token])


def reduce_by_key(postings_lists):
    gens = [(item for item in pl) for pl in postings_lists]
    items = [next(gen) for gen in gens]

    combined = []
    while any(tf != 0 for _, tf in items):
        index, (doc_id, _) = min(enumerate(items), key=lambda x: x[1][0])
        lst = [(_doc_id, _tf) for _doc_id, _tf in items if _doc_id == doc_id]
        combined.append((doc_id, sum(tf for _, tf in lst)))

        for index, item in enumerate(items):
            if item[0] == doc_id:
                try:
                    items[index] = next(gens[index])
                except StopIteration:
                    items[index] = (float('inf'), 0)
    return combined


def query(query: str, bucket_name: str):
    tokens = [token.group() for token in RE_WORD.finditer(query.lower())]
    tokens = [token for token in tokens if token not in all_stopwords]
    token_pl_title = {token: retrieve_posting_list_title(token, bucket_name) for token in tokens}

    bm25_per_term_doc = []
    for token in token_pl_title.keys():
        curr_term = []
        for doc_id, tf in token_pl_title[token]:
            curr_term.append((doc_id, calculate_bm25_per_tf(token, (doc_id, tf), inverted_title)))
        bm25_per_term_doc.append(curr_term)

    bm25_title = reduce_by_key(bm25_per_term_doc)

    return sorted(bm25_title, key=lambda x: x[1], reverse=True)



In [110]:
print(query(query='high sun', bucket_name='bucket324955145'))

[(18193684, 405323.96688466676), (25406470, 405323.96688466676), (26987097, 405323.96688466676), (26987131, 405323.96688466676), (26987167, 405323.96688466676), (26987196, 405323.96688466676), (26987249, 405323.96688466676), (26987268, 405323.96688466676), (26987294, 405323.96688466676), (26987448, 405323.96688466676), (26987467, 405323.96688466676), (26987498, 405323.96688466676), (30865599, 405323.96688466676), (32264549, 405323.96688466676), (35294525, 405323.96688466676), (38889165, 405323.96688466676), (67114306, 405323.96688466676), (54732, 405323.96688466676), (30699330, 405323.96688466676), (19684597, 405323.96688466676), (59684193, 405323.96688466676), (1684074, 405323.96688466676), (35645448, 405323.96688466676), (207646, 405323.96688466676), (3250824, 405323.96688466676), (63678325, 405323.96688466676), (3976275, 405323.96688466676), (30277450, 405323.96688466676), (6213256, 405323.96688466676), (17595583, 405323.96688466676), (57206581, 405323.96688466676), (26238124, 40532

In [ ]:
print(calculate_bm25_per_tf(token='anarchism', doc_id_tf=(12, 1), inverted=inverted_text))
print(calculate_bm25_per_tf(token='anarchism', doc_id_tf=(12, 1), inverted=inverted_text))

405323.9668576603
405323.9668576603


In [ ]:
pls = [
    [(1, 3), (3, 1)],
    [(1, 2), (2, 2), (4, 1)],
    [(2, 3), (3, 3)]
]

In [ ]:
gens = [(item for item in pl) for pl in pls]

In [103]:
# to check if a wikipage exists, and if it does, what is its doc_id


import requests

def get_wikipedia_page_entity_id(title):
    # API endpoint for Wikidata
    wikidata_api_url = "https://www.wikidata.org/w/api.php"

    # Parameters for the API request
    params = {
        "action": "wbgetentities",
        "sites": "enwiki",
        "titles": title,
        "format": "json"
    }

    # Make the API request
    response = requests.get(wikidata_api_url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        # Check if the item exists in Wikidata
        if 'entities' in data:
            entities = data['entities']
            # If the result is not empty, the page exists in Wikidata
            if len(entities) > 0:
                # Return the entity ID (QID) of the page
                entity_id = list(entities.keys())[0]
                return entity_id
    # If there's an error or no entities were found, return None
    return None

# Example usage
title = "Sun"  # Replace with the title of the Wikipedia page you want to check
entity_id = get_wikipedia_page_entity_id(title)
if entity_id:
    print(f"The Wikipedia page '{title}' exists in Wikidata with entity ID '{entity_id}'.")
else:
    print(f"The Wikipedia page '{title}' does not exist in Wikidata.")


The Wikipedia page 'Sun' exists in Wikidata with entity ID 'Q525'.
